# **GROUP 3 F1L Pipeline**

### **Setting -up the Working Directory and the Environment**

In [ ]:
# Start a fresh run
# rm -r "/content/gdrive/MyDrive/MBB_211_CP1/"

In [ ]:
import os
import subprocess

#### **Run this First:**

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')
outdir = "/content/gdrive/MyDrive/MBB_211_CP1/"
os.makedirs(outdir, exist_ok=True)

Mounted at /content/gdrive/


#### **Then mount the Shared GDrive Folder to your personal outdir**

*Shared Working Google Drive Link:* https://drive.google.com/drive/folders/1Ar0JSUZ1gd1uu7rrL5_Nq-E-tI0jTRci?usp=drive_link

To mount the Shared Working Google Drive Link in your $outdir.
- Add the Shared Working Google Drive Link into your working directory as shortcut (folder by folder, the three folder)
  - Right click the Shared Folder, then add the folder to the oudir that you created, "outdir = "/content/gdrive/MyDrive/MBB_211_CP1/"".

#### **Installation of Softwares and Dependencies**



In [ ]:
! apt update

# Single line Installation Command
! apt install -y ncbi-entrez-direct clustalw clustalo iqtree

# Uncomment this accordingly if the installation is unsuccesful.
#! apt install -y ncbi-entrez-direct
#! apt install -y clustalw
#! apt install -y
#! apt-get install iqtree

megadir = "/content/gdrive/MyDrive/MBB_211_CP1/03_Phylo_Tree/mega"
! apt install -y desktop-file-utils libgtk2.0-0
! mega_cc_deb=$megadir/configs/mega-cc_12.0.14-1_amd64_beta.deb
! sudo dpkg -i $mega_cc_deb
#! sudo apt --fix-broken install -y # to fix dependencies if needed uncomment

### **Retrieval and Preprocessing of Sequences:**


#### **Edward's 64 Genes Version**

**1. Retrieve gene metadata Using NCBI Entrez Direct (esearch + efetch):** For each taxon in the list
  * *Gene database* was used instead of Nucleotide database because it contains curated gene-level information, no raw community submissions, and standardized gene annotations.
  * Filtering applied:
    * Only include entries from species-level (sensu stricto) — exclude varieties(var.)
    * For matK: keep only entries with GeneticSource = *chloroplast*
    * For 18S rRNA: keep only entries with GeneticSource = *genomic* (nuclear)
  * Compute gene length = abs(ChrStop - ChrStart)
  * Use a seen[] filter to remove duplicate species entries.
  * Save results (accession, species name, start, stop, length, etc.) to:
    * matk_[taxon].txt
    * 18S_[taxon].txt

In [ ]:
# Install Entrez Direct
! apt install -y ncbi-entrez-direct

In [ ]:
# List of plant taxa groups to search
taxa = ["Bryophyta", "Acrogymnospermae", "Arecaceae", "Bromeliaceae", "Cyperaceae", "Musaceae", "Orchidaceae", "Poaceae", "Typhaceae", "Zingiberaceae", "Brassicaceae", "Gesneriaceae", "Rosaceae", "Solanaceae", "Fabaceae", "Euphorbiaceae", "Betulaceae", "Cannabaceae", "Cornaceae", "Vitaceae", "Lamiaceae", "Amaranthaceae", "Actinidiaceae", "Asteraceae", "Ebenaceae", "Acanthaceae", "Cucurbitaceae", "Fagaceae", "Malvaceae", "Balsaminaceae", "Anacardiaceae", "Juglandaceae", "Myrtaceae", "Celastraceae", "Salicaceae", "Lythraceae", "Convolvulaceae", "Theaceae", "Moraceae"]

# Place categories:
categories = {
    "Bryophytes": ["Bryophyta"],
    "Gymnosperms": ["Acrogymnospermae"],
    "Monocots": ["Arecaceae", "Bromeliaceae", "Cyperaceae", "Musaceae", "Orchidaceae", "Poaceae", "Typhaceae", "Zingiberaceae"],
    "Dicots": ["Brassicaceae", "Gesneriaceae", "Rosaceae", "Solanaceae", "Fabaceae", "Euphorbiaceae", "Betulaceae", "Cannabaceae", "Cornaceae", "Vitaceae", "Lamiaceae", "Amaranthaceae", "Actinidiaceae", "Asteraceae", "Ebenaceae", "Acanthaceae", "Cucurbitaceae", "Fagaceae", "Malvaceae", "Balsaminaceae", "Anacardiaceae", "Juglandaceae", "Myrtaceae", "Celastraceae", "Salicaceae", "Lythraceae", "Convolvulaceae", "Theaceae", "Moraceae"]
}

esearchdir = "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/"
os.makedirs(esearchdir, exist_ok=True)

# Conduct esearch for matk
for taxon in taxa:
  outfile = os.path.join(esearchdir, f"matk_{taxon}.txt")
  print(f"Processing matk in {taxon} -> {outfile}")
  cmd = f"""
  esearch -db gene -query 'matk[gene] AND {taxon}[orgn] NOT "var"' | \
  efetch -format docsum | \
  xtract -pattern DocumentSummary -first ChrAccVer ScientificName ChrStart ChrStop GeneticSource Description -tab "\\t" | \
  awk -F '\\t' '($3 != "" && $4 != "" && $5 == "chloroplast") {{
      len = ($4>$3 ? $4-$3+1 : $3-$4+1);
      if(!seen[$2]++) print $1 "\\t" $2 "\\t" len "\\t" $3 "\\t" $4 "\\t" $5 "\\t" $6
  }}'"""
  with open(outfile, "w") as f:
    subprocess.run(cmd, shell=True, executable="/bin/bash", stdout=f)

# Conduct esearch for 18S rRNA
for taxon in taxa:
  outfile = os.path.join(esearchdir, f"18S_{taxon}.txt")
  print(f"Processing matk in {taxon} -> {outfile}")
  cmd = f"""
  esearch -db gene -query '"18S ribosomal RNA"[All Fields] AND {taxon}[orgn] NOT "var"' | \
  efetch -format docsum | \
  xtract -pattern DocumentSummary -first ChrAccVer ScientificName ChrStart ChrStop GeneticSource Description -tab "\\t" | \
  awk -F '\\t' '($3 != "" && $4 != "" && $5 == "genomic") {{
      len = ($4>$3 ? $4-$3+1 : $3-$4+1);
      if(!seen[$2]++) print $1 "\\t" $2 "\\t" len "\\t" $3 "\\t" $4 "\\t" $5 "\\t" $6
  }}'"""
  with open(outfile, "w") as f:
    subprocess.run(cmd, shell=True, executable="/bin/bash", stdout=f)


In [ ]:
# Sample esearch result
! cat "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/matk_Bryophyta.txt" | head -n 10
! cat "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/18S_Bryophyta.txt" | head -n 10

**2. Identify species present in both gene files:** (they contain both genes)
  * Compare matk_[taxon] with 18S_[taxon].
  * Keep only species present in both files.
  * Save to both_[taxon].txt.
**3. Length filtering:**
  * Keep only species with:
    * *matK length 1500–1600 bp*
    * *18S rRNA length 1700–2000 bp*

In [ ]:
# Check if an organism has both gene sequence and within approximate size [matk=1500-1600] [18S=1700-1800]
MATK_MIN, MATK_MAX = 1500, 1600
RNA_MIN, RNA_MAX = 1700, 2000
for taxon in taxa:
    matK_file = os.path.join(esearchdir, f"matk_{taxon}.txt")
    rRNA_file = os.path.join(esearchdir, f"18S_{taxon}.txt")
    out_file = os.path.join(esearchdir, f"both_{taxon}.txt")
    print(f"Processing {taxon} -> {out_file}")
    matK_dict = {}
    with open(matK_file) as f1:
        for line in f1:
            parts = line.rstrip("\n").split("\t")
            if len(parts) > 2:
                species = parts[1]
                try:
                    size = int(parts[2])
                except ValueError:
                    continue
                if MATK_MIN <= size <= MATK_MAX:
                    if species not in matK_dict:
                        matK_dict[species] = line.rstrip("\n")
    with open(rRNA_file) as f2, open(out_file, "w") as out:
        for line in f2:
            parts = line.rstrip("\n").split("\t")
            if len(parts) > 2:
                species = parts[1]
                try:
                    size = int(parts[2])
                except ValueError:
                    continue

                if species in matK_dict and (RNA_MIN <= size <= RNA_MAX):
                    out.write(matK_dict[species] + "\t" + line)

Processing Bryophyta -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Bryophyta.txt
Processing Acrogymnospermae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Acrogymnospermae.txt
Processing Arecaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Arecaceae.txt
Processing Bromeliaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Bromeliaceae.txt
Processing Cyperaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Cyperaceae.txt
Processing Musaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Musaceae.txt
Processing Orchidaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Orchidaceae.txt
Processing Poaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Poaceae.txt
Processing Typhaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Typhaceae.txt
Processing Zingiberaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Zingiberaceae.txt
Processing Brassicaceae -> /content/gdrive/MyDrive/MBB_211_CP1/esearch/both_Brassicaceae.txt
Processin

In [ ]:
# Sample esearch result (Present matk and 18S)
! cat "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/both_Gunneridae.txt" | wc -l

**4. Select taxonomic representatives:**
  * For each genus → keep only one representative species. From these:
    * Select up to *5 genera* per other major plant family
    * Select *1 genus* from Bryophyta (outgroup)
  * These form the final dataset of organisms.

**5. Retrieve sequence data using accession and coordinates:**
  * For each selected species:
    * Extract *Accession number*, *ChrStart*, *ChrStop*
  * Use efetch + sequence range to download the actual DNA sequences for matK and 18S rRNA
  * Append the added sequence in a FASTA file.

In [ ]:
# Isolate unique genus
summary_file = os.path.join(esearchdir, "unique_genus.txt")
with open(summary_file, "w") as out_f:
    for filename in os.listdir(esearchdir):
        if filename.startswith("both_"):
            filepath = os.path.join(esearchdir, filename)
            unique_first_words = set()
            unique_lines = []
            with open(filepath) as f:
                for line in f:
                    parts = line.rstrip("\n").split("\t")
                    if len(parts) > 1:
                        first_word = parts[1].split()[0]
                        if first_word not in unique_first_words:
                            unique_first_words.add(first_word)
                            unique_lines.append(line)
            print(f"{filename}: {len(unique_first_words)} genus")
            for line in unique_lines:
                out_f.write(f"{filename}\t{line}")

both_Bryophyta.txt: 1 genus
both_Acrogymnospermae.txt: 1 genus
both_Arecaceae.txt: 2 genus
both_Bromeliaceae.txt: 0 genus
both_Cyperaceae.txt: 0 genus
both_Musaceae.txt: 1 genus
both_Orchidaceae.txt: 1 genus
both_Poaceae.txt: 10 genus
both_Typhaceae.txt: 1 genus
both_Zingiberaceae.txt: 1 genus
both_Brassicaceae.txt: 4 genus
both_Gesneriaceae.txt: 2 genus
both_Rosaceae.txt: 3 genus
both_Solanaceae.txt: 4 genus
both_Fabaceae.txt: 9 genus
both_Euphorbiaceae.txt: 4 genus
both_Betulaceae.txt: 1 genus
both_Cannabaceae.txt: 2 genus
both_Cornaceae.txt: 1 genus
both_Vitaceae.txt: 1 genus
both_Lamiaceae.txt: 1 genus
both_Amaranthaceae.txt: 1 genus
both_Actinidiaceae.txt: 1 genus
both_Asteraceae.txt: 2 genus
both_Ebenaceae.txt: 1 genus
both_Acanthaceae.txt: 1 genus
both_Cucurbitaceae.txt: 3 genus
both_Fagaceae.txt: 1 genus
both_Malvaceae.txt: 2 genus
both_Balsaminaceae.txt: 1 genus
both_Anacardiaceae.txt: 1 genus
both_Juglandaceae.txt: 2 genus
both_Myrtaceae.txt: 1 genus
both_Celastraceae.txt: 1 

In [ ]:
! cat "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/unique_genus.txt"

both_Bryophyta.txt	NC_037465.1	Physcomitrium patens	1506	56076	54571	chloroplast	maturase K	NC_037272.2	Physcomitrium patens	1820	5024	3205	genomic	18S ribosomal RNA
both_Acrogymnospermae.txt	NC_010548.1	Cryptomeria japonica	1524	43976	42453	chloroplast	maturase K	NC_081406.1	Cryptomeria japonica	1811	431521093	431522903	genomic	18S ribosomal RNA
both_Arecaceae.txt	NC_017602.1	Elaeis guineensis	1542	3251	1710	chloroplast	maturase K	NC_025997.2	Elaeis guineensis	1810	92039444	92041253	genomic	18S ribosomal RNA
both_Arecaceae.txt	NC_013991.2	Phoenix dactylifera	1545	3282	1738	chloroplast	maturase K	NW_024069927.1	Phoenix dactylifera	1810	17433	15624	genomic	18S ribosomal RNA
both_Musaceae.txt	NC_065468.1	Musa acuminata AAA Group	1536	134097	132562	chloroplast	maturase K	NW_027021363.1	Musa acuminata AAA Group	1810	15293	17102	genomic	18S ribosomal RNA
both_Orchidaceae.txt	NC_037361.1	Dendrobium catenatum	1536	3249	1714	chloroplast	maturase K	NW_021324114.1	Dendrobium catenatum	1813	10693

In [ ]:
# Retrieve Gene ID for MSA
input_file = os.path.join(esearchdir, "unique_genus.txt")
summary_file = os.path.join(esearchdir, "summary_head.txt")
matk_accessions_file = os.path.join(esearchdir, "matk_accessions_only.txt")
rna_accessions_file = os.path.join(esearchdir, "rna_accessions_only.txt")
with open(summary_file, "w") as sum_f, open(matk_accessions_file, "w") as macc_f, open(rna_accessions_file, "w") as racc_f:
    sum_f.write("Source\tScientificName\tmatK_Accession\t18S_Accession\n")
    groups = {}
    with open(input_file) as f:
        for line in f:
            parts = line.rstrip("\n").split("\t")
            if len(parts) < 6:
                continue
            key = parts[0]
            groups.setdefault(key, []).append(line)
    for key, lines in groups.items():
        if "both_Bryophyta.txt" in key:
            selected = lines[:1]
        else:
            selected = lines[:5]
        for line in selected:
            parts = line.rstrip("\n").split("\t")
            matK_acc = parts[1]
            sci_name = parts[2]
            rRNA_acc = parts[8]
            matK_len = parts[3]
            rRNA_len = parts[10]
            matK_st = parts[4]
            matK_end = parts[5]
            rna_st = parts[11]
            rna_end = parts[12]
            sum_f.write(f"{key}\t{sci_name}\t{matK_acc}\t{matK_len}\t{rRNA_acc}\t{rRNA_len}\t{matK_st}\t{matK_end}\t{rna_st}\t{rna_end}\n")
            macc_f.write(f"{matK_acc}\t{matK_st}\t{matK_end}\n")
            racc_f.write(f"{rRNA_acc}\t{rna_st}\t{rna_end}\n")
print("Done! Output saved in:")
print(summary_file)
print(matk_accessions_file)
print(rna_accessions_file)

Done! Output saved in:
/content/gdrive/MyDrive/MBB_211_CP1/esearch/summary_head.txt
/content/gdrive/MyDrive/MBB_211_CP1/esearch/matk_accessions_only.txt
/content/gdrive/MyDrive/MBB_211_CP1/esearch/rna_accessions_only.txt


In [ ]:
# Gene ID and gene lengths of the final dataset
! cat "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/summary_head.txt"

Source	ScientificName	matK_Accession	18S_Accession
both_Bryophyta.txt	Physcomitrium patens	NC_037465.1	1506	NC_037272.2	1820	56076	54571	5024	3205
both_Acrogymnospermae.txt	Cryptomeria japonica	NC_010548.1	1524	NC_081406.1	1811	43976	42453	431521093	431522903
both_Arecaceae.txt	Elaeis guineensis	NC_017602.1	1542	NC_025997.2	1810	3251	1710	92039444	92041253
both_Arecaceae.txt	Phoenix dactylifera	NC_013991.2	1545	NW_024069927.1	1810	3282	1738	17433	15624
both_Musaceae.txt	Musa acuminata AAA Group	NC_065468.1	1536	NW_027021363.1	1810	134097	132562	15293	17102
both_Orchidaceae.txt	Dendrobium catenatum	NC_037361.1	1536	NW_021324114.1	1813	3249	1714	10693	8881
both_Poaceae.txt	Lolium perenne	NC_009950.1	1536	NC_067246.2	1808	3249	1714	120147741	120145934
both_Poaceae.txt	Miscanthus floridulus	NC_035750.1	1548	NW_027098526.1	1811	3219	1672	20732	18922
both_Poaceae.txt	Phragmites australis	NC_022958.1	1542	NW_026909971.1	1811	3213	1672	15457	13647
both_Poaceae.txt	Triticum urartu	NC_021762.1	1

In [ ]:
# Gene ID and gene lengths of the final dataset
! cat "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/matk_accessions_only.txt" | head -n 5
! cat "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/rna_accessions_only.txt" | head -n 5

**MSA Preprocessing**

**6. Retrieval of FASTA sequence for matk and 18s**

In [ ]:
# Retrieve FASTA for all matk accessions listed
edward_msadir = "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/64_genes_version"
os.makedirs(edward_msadir, exist_ok=True)
out_fasta = os.path.join(edward_msadir, "matk_sequences.fasta")
open(out_fasta, "w").close()
def get_sequence_from_acc(acc, start=None, end=None):
    """Retrieve sequence (no header, 1 line) using Entrez Direct with optional coordinates."""
    seq_range = ""
    if start and end:
        seq_range = f"-seq_start {start} -seq_stop {end}"
    cmd = f'efetch -db nucleotide -id "{acc}" {seq_range} -format fasta'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.stdout:
        seq = "".join([line.strip() for line in result.stdout.splitlines() if not line.startswith(">")])
        return seq
    return None
with open(summary_file) as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) < 6:
            continue
        sci_name = parts[1]
        acc1, start1, end1 = parts[2], parts[6], parts[7]
        header = sci_name.replace(" ", "_")
        print(f"Processing {header} → {acc1}:{start1}-{end1}")
        seq1 = get_sequence_from_acc(acc1, start1, end1)
        if not seq1:
            print(f"Skipping {header} — missing sequence")
            continue
        with open(out_fasta, "a") as fout:
            fout.write(f">{header}\n{seq1}\n")

# Retrieve FASTA for all 18S accessions listed
out_fasta = os.path.join(edward_msadir, "rna_sequences.fasta")
open(out_fasta, "w").close()
def get_sequence_from_acc(acc, start=None, end=None):
    """Retrieve sequence (no header, 1 line) using Entrez Direct with optional coordinates."""
    seq_range = ""
    if start and end:
        seq_range = f"-seq_start {start} -seq_stop {end}"
    cmd = f'efetch -db nucleotide -id "{acc}" {seq_range} -format fasta'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.stdout:
        seq = "".join([line.strip() for line in result.stdout.splitlines() if not line.startswith(">")])
        return seq
    return None
with open(summary_file) as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) < 6:
            continue
        sci_name = parts[1]
        acc1, start1, end1 = parts[4], parts[8], parts[9]
        header = sci_name.replace(" ", "_")
        print(f"Processing {header} → {acc1}:{start1}-{end1}")
        seq1 = get_sequence_from_acc(acc1, start1, end1)
        if not seq1:
            print(f"Skipping {header} — missing sequence")
            continue
        with open(out_fasta, "a") as fout:
            fout.write(f">{header}\n{seq1}\n")

Processing Physcomitrium_patens → NC_037465.1:56076-54571
Processing Cryptomeria_japonica → NC_010548.1:43976-42453
Processing Elaeis_guineensis → NC_017602.1:3251-1710
Processing Phoenix_dactylifera → NC_013991.2:3282-1738
Processing Musa_acuminata_AAA_Group → NC_065468.1:134097-132562
Processing Dendrobium_catenatum → NC_037361.1:3249-1714
Processing Lolium_perenne → NC_009950.1:3249-1714
Processing Miscanthus_floridulus → NC_035750.1:3219-1672
Processing Phragmites_australis → NC_022958.1:3213-1672
Processing Triticum_urartu → NC_021762.1:3206-1668
Processing Hordeum_vulgare_subsp._vulgare → NC_008590.1:3740-2205
Processing Typha_latifolia → NC_013823.1:3225-1684
Processing Zingiber_officinale → NC_044775.1:3300-1753
Processing Arabidopsis_thaliana → NC_000932.1:3569-2055
Processing Raphanus_sativus → NC_024469.1:3567-1993
Processing Brassica_napus → NC_016734.1:3605-2031
Processing Eutrema_salsugineum → NC_028170.1:3646-2066
Processing Primulina_huaijiensis → NC_036413.1:3544-2012


In [ ]:
# Count no. of matk sequences
! grep ">" /content/gdrive/MyDrive/MBB_211_CP1/02_MSA/matk_sequences.fasta | wc -l

In [ ]:
# Length of matk
! awk '/^>/ { if(seq) print length(seq); print; seq="" } !/^>/ { seq = seq $0 } END { if(seq) print length(seq) }' "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/matk_sequences.fasta" | head -n 10

In [ ]:
# Count no. of 18S sequences
! grep ">" /content/gdrive/MyDrive/MBB_211_CP1/02_MSA/rna_sequences.fasta | wc -l

In [ ]:
# Length of 18S
! awk '/^>/ { if(seq) print length(seq); print; seq="" } !/^>/ { seq = seq $0 } END { if(seq) print length(seq) }' "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/rna_sequences.fasta" | head -n 10

**7. Concatenate matK + 18S per Species:**
  * For each species:
  * Concatenate its matK and 18S sequences.
  * Append the added sequence in a FASTA file.

In [ ]:
# Retrieve FASTA for matk and 18S accessions listed and concatenate
out_fasta = os.path.join(edward_msadir, "concatenated_sequences.fasta")
open(out_fasta, "w").close()
def get_sequence_from_acc(acc, start=None, end=None):
    """Retrieve sequence (no header, 1 line) using Entrez Direct with optional coordinates."""
    seq_range = ""
    if start and end:
        seq_range = f"-seq_start {start} -seq_stop {end}"
    cmd = f'efetch -db nucleotide -id "{acc}" {seq_range} -format fasta'
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if result.stdout:
        seq = "".join([line.strip() for line in result.stdout.splitlines() if not line.startswith(">")])
        return seq
    return None
with open(summary_file) as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) < 6:
            continue
        sci_name = parts[1]
        acc1, start1, end1 = parts[2], parts[6], parts[7]
        acc2, start2, end2 = parts[4], parts[8], parts[9]
        header = sci_name.replace(" ", "_")
        print(f"Processing {header}")
        seq1 = get_sequence_from_acc(acc1, start1, end1)
        seq2 = get_sequence_from_acc(acc2, start2, end2)
        if not seq1 or not seq2:
            print(f"Skipping {header} — missing sequence")
            continue
        concatenated = seq1 + seq2
        with open(out_fasta, "a") as fout:
            fout.write(f">{header}\n{concatenated}\n")

Processing Physcomitrium_patens
Processing Cryptomeria_japonica
Processing Elaeis_guineensis
Processing Phoenix_dactylifera
Processing Musa_acuminata_AAA_Group
Processing Dendrobium_catenatum
Processing Lolium_perenne
Processing Miscanthus_floridulus
Processing Phragmites_australis
Processing Triticum_urartu
Processing Hordeum_vulgare_subsp._vulgare
Processing Typha_latifolia
Processing Zingiber_officinale
Processing Arabidopsis_thaliana
Processing Raphanus_sativus
Processing Brassica_napus
Processing Eutrema_salsugineum
Processing Primulina_huaijiensis
Processing Henckelia_pumila
Processing Malus_domestica
Processing Pyrus_communis
Processing Rosa_rugosa
Processing Nicotiana_tomentosiformis
Processing Solanum_lycopersicum
Processing Lycium_barbarum
Processing Capsicum_annuum
Processing Phaseolus_vulgaris
Processing Lotus_japonicus
Processing Prosopis_cineraria
Processing Vigna_angularis
Processing Glycine_max
Processing Euphorbia_lathyris
Processing Hevea_brasiliensis
Processing Ricin

In [ ]:
# Count no. of concatenated sequences
! grep ">" /content/msa/concatenated_sequences.fasta | wc -l

In [ ]:
# Length of matk + 18S
! awk '/^>/ { if(seq) print length(seq); print; seq="" } !/^>/ { seq = seq $0 } END { if(seq) print length(seq) }' "/content/msa/concatenated_sequences.fasta" | head -n 10

There are currently three FASTA files for downstream MSA:
  * **matk_msa.fasta**
  * **18S_msa.fasta**
  * **concatenated_msa.fasta**

In [ ]:
# Count how many per category
counts = {key: 0 for key in categories}
dicot_family_counts = {fam: 0 for fam in categories["Dicots"]}
with open(summary_file) as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) < 2:
            continue
        raw_taxon = parts[0]
        taxon = raw_taxon.replace("both_", "").replace(".txt", "")
        for cat, taxa_list in categories.items():
            if any(t == taxon for t in taxa_list):
                counts[cat] += 1
                if cat == "Dicots":
                    dicot_family_counts[taxon] += 1
                break
print("Category counts:", counts)
print("Dicot family counts:", dicot_family_counts)

Category counts: {'Bryophytes': 1, 'Gymnosperms': 1, 'Monocots': 11, 'Dicots': 51}
Dicot family counts: {'Brassicaceae': 4, 'Gesneriaceae': 2, 'Rosaceae': 3, 'Solanaceae': 4, 'Fabaceae': 5, 'Euphorbiaceae': 4, 'Betulaceae': 1, 'Cannabaceae': 2, 'Cornaceae': 1, 'Vitaceae': 1, 'Lamiaceae': 1, 'Amaranthaceae': 1, 'Actinidiaceae': 1, 'Asteraceae': 2, 'Ebenaceae': 1, 'Acanthaceae': 1, 'Cucurbitaceae': 3, 'Fagaceae': 1, 'Malvaceae': 2, 'Balsaminaceae': 1, 'Anacardiaceae': 1, 'Juglandaceae': 2, 'Myrtaceae': 1, 'Celastraceae': 1, 'Salicaceae': 1, 'Lythraceae': 1, 'Convolvulaceae': 1, 'Theaceae': 1, 'Moraceae': 1}


#### **Czar's 21 Genes Version**



In [23]:
# --- Directory Setup ---
czar_21_genes_dir = "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/manual"

czar_21_18s_dir = f"{czar_21_genes_dir}/18S"
czar_21_matK_dir = f"{czar_21_genes_dir}/matK"

czar_21_18s_var = f"{czar_21_genes_dir}/merged_18s.fasta"
czar_21_matK_var = f"{czar_21_genes_dir}/merged_matK.fasta"
czar_21_concatenated_var = f"{czar_21_genes_dir}/merged_18S_+_matK.fasta"

! touch $czar_21_18s_var
! touch $czar_21_matK_var
! touch $czar_21_concatenated_var

# --- Merging FASTA files (recursively) ---
import os

def merge_fastas_recursive(input_dir, output_file):
    """
    Recursively find all FASTA (.fasta, .fa) files inside a directory
    and merge them into one output FASTA file.
    """
    fasta_files = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.fasta', '.fa')):
                fasta_files.append(os.path.join(root, file))

    with open(output_file, 'w') as outfile:
        for fasta in sorted(fasta_files):
            with open(fasta) as infile:
                outfile.write(infile.read().strip() + "\n")
    print(f"✅ Merged {len(fasta_files)} FASTA files (recursive) into {output_file}")


# --- Perform merging for 18S and matK ---
merge_fastas_recursive(czar_21_18s_dir, czar_21_18s_var)
merge_fastas_recursive(czar_21_matK_dir, czar_21_matK_var)


# --- Concatenate merged 18S and matK FASTAs ---
def concatenate_fastas(fasta1, fasta2, output_file):
    """
    Concatenate two FASTA files into a single output FASTA.
    This simply appends one after the other.
    """
    with open(output_file, 'w') as outfile:
        for f in [fasta1, fasta2]:
            with open(f) as infile:
                outfile.write(infile.read().strip() + "\n")
    print(f"✅ Concatenated {os.path.basename(fasta1)} and {os.path.basename(fasta2)} → {output_file}")

concatenate_fastas(czar_21_18s_var, czar_21_matK_var, czar_21_concatenated_var)


✅ Merged 21 FASTA files (recursive) into /content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/manual/merged_18s.fasta
✅ Merged 21 FASTA files (recursive) into /content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/manual/merged_matK.fasta
✅ Concatenated merged_18s.fasta and merged_matK.fasta → /content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/manual/merged_18S_+_matK.fasta


#### **Curated_21_Genes_Version (Working on this, still empty)**

In [ ]:
# --- Directory Setup ---
curated_21_dir = "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/curated_21_genes_version"

curated_21_28s_dir = f"{curated_21_dir}/28s"
curated_21_matK_dir = f"{curated_21_dir}/matK"

curated_21_28s_var = f"{curated_21_dir}/merged_18s.fasta"
curated_21_matK_var = f"{curated_21_dir}/merged_matK.fasta"
curated_21_concatenated_var = f"{curated_21_dir}/merged_28S_+_matK.fasta"


# --- Merging FASTA files (recursive) ---
import os

def merge_fastas_recursive(input_dir, output_file):
    """
    Recursively find all FASTA (.fasta, .fa) files inside a directory
    and merge them into one output FASTA file.
    """
    fasta_files = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.fasta', '.fa')):
                fasta_files.append(os.path.join(root, file))

    with open(output_file, 'w') as outfile:
        for fasta in sorted(fasta_files):
            with open(fasta) as infile:
                outfile.write(infile.read().strip() + "\n")
    print(f"✅ Merged {len(fasta_files)} FASTA files (recursive) into {output_file}")


# --- Perform merging for 28S and matK ---
merge_fastas_recursive(curated_21_28s_dir, curated_21_28s_var)
merge_fastas_recursive(curated_21_matK_dir, curated_21_matK_var)


# --- Concatenate merged 28S and matK FASTAs ---
def concatenate_fastas(fasta1, fasta2, output_file):
    """
    Concatenate two FASTA files into a single output FASTA.
    This simply appends one after the other.
    """
    with open(output_file, 'w') as outfile:
        for f in [fasta1, fasta2]:
            with open(f) as infile:
                outfile.write(infile.read().strip() + "\n")
    print(f"✅ Concatenated {os.path.basename(fasta1)} and {os.path.basename(fasta2)} → {output_file}")

concatenate_fastas(curated_21_28s_var, curated_21_matK_var, curated_21_concatenated_var)


### **Multiple Sequence Alignment**

In [7]:
# Install ClustalW
! apt install -y clustalw clustalo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libargtable2-0
Suggested packages:
  clustalx seaview
The following NEW packages will be installed:
  clustalo clustalw libargtable2-0
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 548 kB of archives.
After this operation, 1,512 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libargtable2-0 amd64 13-1.1 [14.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 clustalo amd64 1.2.4-7 [259 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 clustalw amd64 2.1+lgpl-7 [275 kB]
Fetched 548 kB in 1s (648 kB/s)
Selecting previously unselected package libargtable2-0.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../libargtable2-0_13-1.1_amd64.deb ...
Unpacking libargtable2-0 (13-1.1) ...
Sel

In [ ]:
# Install ClustalO
! apt install -y clustalo

In [ ]:
! nproc

#### **MSA using ClustalW and ClustalO**

##### **Edward's 64 Genes Version**

In [ ]:
edward_msadir = "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/64_genes_version"

# Align sequences using Clustal W
! clustalw -infile="$edward_msadir/matk_sequences_ClustalW.fasta" -type=dna -output=fasta -outfile="$edward_msadir/msa_matk_ClustalW.fasta"
! clustalw -infile="$edward_msadir/rna_sequences_ClustalW.fasta" -type=dna -output=fasta -outfile="$edward_msadir/msa_rna_ClustalW.fasta"
! clustalw -infile="$edward_msadir/concatenated_sequences_ClustalW.fasta" -type=dna -output=fasta -outfile="$edward_msadir/msa_concatenated_ClustalW.fasta"

# Align sequences using Clustal O
edward_msadir="/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/64_genes_version"
clustalo -i "$edward_msadir/matk_sequences.fasta" -o "$edward_msadir/msa_matk_ClustalO.fasta" --seqtype=DNA --output-order=tree-order --force --full --full-iter --iterations=1000 --percent-id --threads=$(nproc)
clustalo -i "$edward_msadir/rna_sequences.fasta" -o "$edward_msadir/msa_rna_ClustalO.fasta" --seqtype=DNA --output-order=tree-order --force --full --full-iter --iterations=1000 --percent-id --threads=$(nproc)
clustalo -i "$edward_msadir/concatenated_sequences.fasta" -o "$edward_msadir/msa_concatenated_ClustalO.fasta" --seqtype=DNA --output-order=tree-order --force --full --full-iter --iterations=1000 --percent-id --threads=$(nproc)


Streaming output truncated to the last 5000 lines.
Sequences (30:33) Aligned. Score:  76
Sequences (30:34) Aligned. Score:  75
Sequences (30:35) Aligned. Score:  76
Sequences (30:36) Aligned. Score:  77
Sequences (30:37) Aligned. Score:  78
Sequences (30:38) Aligned. Score:  77
Sequences (30:39) Aligned. Score:  75
Sequences (30:40) Aligned. Score:  75
Sequences (30:41) Aligned. Score:  71
Sequences (30:42) Aligned. Score:  73
Sequences (30:43) Aligned. Score:  75
Sequences (30:44) Aligned. Score:  73
Sequences (30:45) Aligned. Score:  73
Sequences (30:46) Aligned. Score:  76
Sequences (30:47) Aligned. Score:  70
Sequences (30:48) Aligned. Score:  77
Sequences (30:49) Aligned. Score:  76
Sequences (30:50) Aligned. Score:  76
Sequences (30:51) Aligned. Score:  75
Sequences (30:52) Aligned. Score:  74
Sequences (30:53) Aligned. Score:  75
Sequences (30:54) Aligned. Score:  72
Sequences (30:55) Aligned. Score:  74
Sequences (30:56) Aligned. Score:  76
Sequences (30:57) Aligned. Score:  77

In [ ]:
# Check if okay
! head -n 10 "$edward_msadir/msa_matk_ClustalW.fasta"
! head -n 10 "$edward_msadir/msa_rna_ClustalW.fasta"
! head -n 10 "$edward_msadir/msa_concatenated_ClustalW.fasta"
! head -n 10 "$edward_msadir/msa_matk_Clustal_O.fasta"
! head -n 10 "$edward_msadir/msa_rna_Clustal_O.fasta"
! head -n 10 "$edward_msadir/msa_concatenated_Clustal_O.fasta"

##### **Czar's 21 Genes Version: MSA**

In [ ]:
# === Directory Setup ===
czar_msadir = "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/manual"
!mkdir -p "$czar_msadir"

# === Input FASTA Files ===
czar_21_genes_dir = "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/manual"
!ls -l "$czar_21_genes_dir"

czar_21_18s_var = f"{czar_21_genes_dir}/merged_18s.fasta"
czar_21_matK_var = f"{czar_21_genes_dir}/merged_matK.fasta"
czar_21_concatenated_var = f"{czar_21_genes_dir}/merged_18S_+_matK.fasta"

# === Multiple Sequence Alignment using ClustalW ===
!clustalw -INFILE="$czar_21_matK_var" -TYPE=DNA -OUTPUT=FASTA -OUTFILE="$czar_msadir/msa_matk_ClustalW.fasta"
!clustalw -INFILE="$czar_21_18s_var" -TYPE=DNA -OUTPUT=FASTA -OUTFILE="$czar_msadir/msa_rna_ClustalW.fasta"
!clustalw -INFILE="$czar_21_concatenated_var" -TYPE=DNA -OUTPUT=FASTA -OUTFILE="$czar_msadir/msa_concatenated_ClustalW.fasta"

# === Multiple Sequence Alignment using Clustal Omega ===
!clustalo -i "$czar_21_matK_var" -o "$czar_msadir/msa_matk_ClustalO.fasta" --seqtype=DNA --output-order=tree-order --force --full --full-iter --iterations=1000 --percent-id --threads=$(nproc)
!clustalo -i "$czar_21_18s_var" -o "$czar_msadir/msa_rna_ClustalO.fasta" --seqtype=DNA --output-order=tree-order --force --full --full-iter --iterations=1000 --percent-id --threads=$(nproc)
!clustalo -i "$czar_21_concatenated_var" -o "$czar_msadir/msa_concatenated_ClustalO.fasta" --seqtype=DNA --output-order=tree-order --force --full --full-iter --iterations=1000 --percent-id --threads=$(nproc)


total 161
drwx------ 7 root root  4096 Oct 20 17:01 18S
drwx------ 2 root root  4096 Oct 22 08:50 18S_+_matK
drwx------ 7 root root  4096 Oct 20 17:01 matK
-rw------- 1 root root 40455 Oct 22 09:48 merged_18s.fasta
-rw------- 1 root root 74787 Oct 22 09:48 merged_18S_+_matK.fasta
-rw------- 1 root root   858 Oct 22 09:38 merged_matK.dnd
-rw------- 1 root root 34332 Oct 22 09:48 merged_matK.fasta



 CLUSTAL 2.1 Multiple Sequence Alignments


Sequence type explicitly set to DNA
Sequence format is Pearson
Sequence 1: NC_000932.1_2056-3570   1515 bp
Sequence 2: NC_016734.1_2032-3606   1575 bp
Sequence 3: MT430983.1_1951-3531    1581 bp
Sequence 4: NC_024469.1_1994-3568   1575 bp
Sequence 5: NC_047402.1_2087-3598   1512 bp
Sequence 6: NC_037358.1_2088-3617   1530 bp
Sequence 7: NC_007942.1_2011-3528   1518 bp
Sequence 8: NC_018051.1_4972-6486   1515 bp
Sequence 9: KY126308.1_54572-56077  1506 bp
Sequence 10: KX249805.1_11749-13278  1530 bp
Sequence 11: AF143437.1_342-1896     1555 bp
Seque

### **Phylogenetic Tree Construction**

In [18]:
# Install IQTREE
! apt-get install iqtree

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  iqtree
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 3,776 kB of archives.
After this operation, 23.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 iqtree amd64 2.0.7+dfsg-1 [3,776 kB]
Fetched 3,776 kB in 0s (9,474 kB/s)
Selecting previously unselected package iqtree.
(Reading database ... 126705 files and directories currently installed.)
Preparing to unpack .../iqtree_2.0.7+dfsg-1_amd64.deb ...
Unpacking iqtree (2.0.7+dfsg-1) ...
Setting up iqtree (2.0.7+dfsg-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
megadir = "/content/gdrive/MyDrive/MBB_211_CP1/03_Phylo_Tree/mega"
! apt update
! apt install -y desktop-file-utils libgtk2.0-0
! mega_cc_deb=$megadir/configs/mega-cc_12.0.14-1_amd64_beta.deb
! sudo dpkg -i $mega_cc_deb
! sudo apt --fix-broken install -y # to fix dependencies if needed uncomment

#### **Running Tree Inference using IQTREE**

##### **Edward's 64 Genes Version: IQTREE2**

In [ ]:
# If run time of iqtree disconnected uncomment this:
esearchdir = "/content/gdrive/MyDrive/MBB_211_CP1/01_Raw_Data/esearch/"
summary_file = os.path.join(esearchdir, "summary_head.txt")
# Then run the code below.

In [ ]:
# Set Bryophyte for rooting
bryophyta_species = []
with open(summary_file) as f:
    for line in f:
        if "Bryophyta" in line:
            parts = line.strip().split("\t")
            if len(parts) > 1:
                sci_name = parts[1]
                header = sci_name.replace(" ", "_")
                bryophyta_species.append(header)
print(bryophyta_species)

['Physcomitrium_patens']


In [ ]:
# Create tree files
iqtreedir = "/content/gdrive/MyDrive/MBB_211_CP1/03_Phylo_Tree/iqtree/"
edward_msadir = "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/64_genes_vesion"
os.makedirs(iqtreedir, exist_ok=True)

outgroup = ",".join(bryophyta_species)
! iqtree2 -s "$edward_msadir/msa_matk.fasta" -pre "$iqtreedir/tree_matk" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}
! iqtree2 -s "$edward_msadir/msa_rna.fasta" -pre "$iqtreedir/tree_18S" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}
! iqtree2 -s "$edward_msadir/msa_concatenated.fasta" -pre "$iqtreedir/tree_concatenated" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}

Streaming output truncated to the last 5000 lines.
Estimate model parameters (epsilon = 0.100)
UPDATE BEST LOG-LIKELIHOOD: -17730.904
Iteration 120 / LogL: -17737.386 / Time: 0h:0m:51s (0h:0m:7s left)
Iteration 130 / LogL: -17735.791 / Time: 0h:0m:53s (0h:0m:2s left)
TREE SEARCH COMPLETED AFTER 138 ITERATIONS / Time: 0h:0m:56s

--------------------------------------------------------------------
|                    FINALIZING TREE SEARCH                        |
--------------------------------------------------------------------
Performs final model parameters optimization
Estimate model parameters (epsilon = 0.010)
1. Initial log-likelihood: -17730.904
2. Current log-likelihood: -17730.865
3. Current log-likelihood: -17730.801
4. Current log-likelihood: -17730.742
5. Current log-likelihood: -17730.686
6. Current log-likelihood: -17730.637
7. Current log-likelihood: -17730.590
8. Current log-likelihood: -17730.557
9. Current log-likelihood: -17730.547
Optimal log-likelihood: -17730.5

##### **Czar's 21 Genes Version: IQTREE2**


In [ ]:
# Create tree files
czar_iqtreedir = "/content/gdrive/MyDrive/MBB_211_CP1/03_Phylo_Tree/czar_iqtree/"
czar_msadir = "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/manual"
os.makedirs(czar_iqtreedir, exist_ok=True)

! iqtree2 -s "$czar_msadir/msa_matk_ClustalW.fasta" -pre "$czar_iqtreedir/tree_matk_ClustalW" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}
! iqtree2 -s "$czar_msadir/msa_rna_ClustalW.fasta" -pre "$czar_iqtreedir/tree_18S_ClustalW" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}
! iqtree2 -s "$czar_msadir/msa_concatenated_ClustalW.fasta" -pre "$czar_iqtreedir/tree_concatenated_ClustalW" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}

! iqtree2 -s "$czar_msadir/msa_matk_ClustalO.fasta" -pre "$czar_iqtreedir/tree_matk_ClustalO" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}
! iqtree2 -s "$czar_msadir/msa_rna_ClustalO.fasta" -pre "$czar_iqtreedir/tree_18S_ClustalO" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}
! iqtree2 -s "$czar_msadir/msa_concatenated_ClustalO.fasta" -pre "$czar_iqtreedir/tree_concatenated_ClustalO" -m MFP -b 1000 -alrt 1000 -abayes -o {outgroup}

#### **Running Tree Inference using MEGA**




##### **Edward's 64 Genes Version: MEGA12CC**

In [ ]:
# Setting Directories
edward_msadir = "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/64_genes_vesion"
edward_megadir = "/content/gdrive/MyDrive/MBB_211_CP1/03_Phylo_Tree/mega/64_genes_vesion"
mega_mao_config="$megadir/configs/infer_ML_nucleotide.mao"
os.makedirs(edward_megadir, exist_ok=True)

# Running MEGA CC
! megacc -a $mega_mao_config -d $edward_msadir/msa_matk_ClustalO.fasta -o $edward_megadir/matk
! megacc -a $mega_mao_config -d $edward_msadir/msa_rna_ClustalO.fasta -o $edward_megadir/rna
! megacc -a $mega_mao_config -d $edward_msadir/msa_concatenated_ClustalO.fasta -o $edward_megadir/concatenated

! megacc -a $mega_mao_config -d $edward_msadir/msa_matk_ClustalW.fasta -o $edward_megadir/matk
! megacc -a $mega_mao_config -d $edward_msadir/msa_rna_ClustalW.fasta -o $edward_megadir/rna
! megacc -a $mega_mao_config -d $edward_msadir/msa_concatenated_ClustalW.fasta -o $edward_megadir/concatenated

In [ ]:
! head -n 2 /content/gdrive/MyDrive/MBB_211_CP1/mega/matk.csv
! head -n 2 /content/gdrive/MyDrive/MBB_211_CP1/mega/rna.csv
! head -n 2 /content/gdrive/MyDrive/MBB_211_CP1/mega/concatenated.csv

##### **Czar's 21 Genes Version: MEGA12CC**


In [ ]:
# Setting Directories
czar_msadir = "/content/gdrive/MyDrive/MBB_211_CP1/02_MSA/manual"
czar_megadir = "/content/gdrive/MyDrive/MBB_211_CP1/03_Phylo_Tree/mega/manual"
mega_mao_config="$megadir/configs/infer_ML_nucleotide.mao"
os.makedirs(edward_megadir, exist_ok=True)

# Running MEGA CC
! megacc -a $mega_mao_config -d $czar_msadir/msa_matk_ClustalO.fasta -o $czar_megadir/matk
! megacc -a $mega_mao_config -d $czar_msadir/msa_rna_ClustalO.fasta -o $czar_megadir/rna
! megacc -a $mega_mao_config -d $czar_msadir/msa_concatenated_ClustalO.fasta -o $czar_megadir/concatenated

! megacc -a $mega_mao_config -d $czar_msadir/msa_matk_ClustalW.fasta -o $czar_megadir/matk
! megacc -a $mega_mao_config -d $czar_msadir/msa_rna_ClustalW.fasta -o $czar_megadir/rna
! megacc -a $mega_mao_config -d $czar_msadir/msa_concatenated_ClustalW.fasta -o $czar_megadir/concatenated

##### **Curated 21 Genes Version: MEGA12 CC**

In [ ]:
# Insert Code Here

**Sample Visualization**

In [ ]:
# Install ETE3
! pip install ete3
! pip install PyQt5
from ete3 import Tree

In [ ]:
# Visualize matK
t = Tree("/content/gdrive/MyDrive/MBB_211_CP1/tree/tree_matk.treefile", format=1, quoted_node_names=True)
print(t.get_ascii(show_internal=True))

In [ ]:
# Visualize 18S
t = Tree("/content/gdrive/MyDrive/MBB_211_CP1/tree/tree_18S.treefile", format=1, quoted_node_names=True)
print(t.get_ascii(show_internal=True))

In [ ]:
# Visualize concatenated
t = Tree("/content/gdrive/MyDrive/MBB_211_CP1/tree/tree_concatenated.treefile", format=1, quoted_node_names=True)
print(t.get_ascii(show_internal=True))

In [ ]:
from google.colab import files
files.download("/content/gdrive/MyDrive/MBB_211_CP1/tree/tree_matk.treefile")
files.download("/content/gdrive/MyDrive/MBB_211_CP1/tree/tree_18S.treefile")
files.download("/content/gdrive/MyDrive/MBB_211_CP1/tree/tree_concatenated.treefile")